In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append("./subjects/")
import pickle
import matplotlib.pyplot as plt
import itertools
from sklearn.cluster import KMeans
import tensorflow as tf
import time
import random, math
import tensorflow_probability as tfb
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics.pairwise import  euclidean_distances
from Utils_Functions import KLdivergence
from sklearn.feature_selection import SelectKBest, SelectFpr,SelectPercentile 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
from aif360.sklearn.metrics import equal_opportunity_difference,average_odds_difference
from Utils_Functions import generate_dataset, eod
import glob
import re
input()
def generate_dataset_shift(data, graph, edges, sens_index,priv_group, unpriv_group):

    dataset_types = [str(data[i].dtype) for i in data.columns]
    succ_generated = 0
    generation_coef = 1
    graph_dic ={}
    for i in graph.columns[1:]:
        if np.where(graph[i])[0].shape[0]==0:
            graph_dic[i]=None
        else:
            graph_dic[i]= graph['Unnamed: 0'][np.where(graph[i])[0]].values

    final_df = pd.DataFrame(columns = data.columns) 
    trial = 0
    not_interesting = False
    while final_df.shape[0]<data.shape[0]:

        if trial > 20:
            not_interesting = True
            return None , False
        df_new_dic ={}
        for edge in graph.sum().index[np.where(graph.sum()==0)[0]]:
            df_new_dic[edge] = np.random.choice(np.unique(data[edge]), size = data.shape[0]*generation_coef) 

        statring_atts = graph.sum().index[np.where(graph.sum()==0)[0]]
        
        while statring_atts.shape[0] != graph['Unnamed: 0'].shape[0]:
            
            for att in graph_dic.keys():
                if att not in statring_atts:
                    if 0 in  [1 if graph_dic[att][i] in statring_atts else 0 for i in range(graph_dic[att].shape[0])]:

                        continue
                    else:
                        edge_logits = 0
                        
                        for cause in graph_dic[att]:
                            edge_logits += (edges[cause+att] * df_new_dic[cause])
                        if np.unique(data[att]).shape[0]==2:
                            df_new_dic[att] =  tfb.distributions.Bernoulli(logits=edge_logits + edges[att+'0'] ).sample().numpy()
                        elif 'float' in dataset_types[np.where(data.columns==att)[0][0]]:

                            df_new_dic[att] =  tfb.distributions.Normal(loc=(edge_logits+ edges[att+'0']), scale= edges['sigma_h']).sample().numpy()
                        
                        else:    
                            df_new_dic[att] =  tfb.distributions.Poisson(rate=tf.exp(edge_logits+ edges[att+'0']) ).sample().numpy()  

                        statring_atts = np.append(statring_atts,att) 

        new_df = pd.DataFrame(columns = data.columns)
        for col in new_df.columns:
            new_df[col] = df_new_dic[col]
        
        ind_inf = np.unique(np.where(new_df>data.max())[0])
        new_df.drop(ind_inf,axis=0,inplace=True)
        for col in range(new_df.columns.shape[0]):
            new_df[new_df.columns[col]]=new_df[new_df.columns[col]].astype(dataset_types[col])
        if new_df.shape[0]<1:
            return None , False
        X2 = new_df.to_numpy()[:,:-1]
        Y2 = new_df.to_numpy()[:,-1]
        #dist = euclidean_distances(X2, centroids)
        #succ_generated += new_df.iloc[np.where((ave_dist>=dist).sum(1)>0)].shape[0]
        final_df = pd.concat([final_df,new_df]).reset_index(drop=True)
        final_df = final_df.drop_duplicates()


            
        trial += 1
   
    #final_df = final_df.astype(int)
    Y2 = final_df.to_numpy()[:,-1]
    if (Y2.sum()/Y2.shape[0]< 0.06) or (Y2.sum()/Y2.shape[0]> 0.95):
        return None,  False
    if priv_group not in final_df[final_df.columns[sens_index]].values or unpriv_group not in final_df[final_df.columns[sens_index]].values:
        return None,  False

    return final_df, True

#dataset ='Bank'
for dataset in ['Adult']:
    if dataset == 'Adult':
        sens_index = 7
        priv_group = 1
        unpriv_group = 0
        data_file_name = 'adult_org-Copy1.csv'
        alg_list = ['ges','simy']
        
    if dataset == 'Compas':
        sens_index = 1
        priv_group = 1
        unpriv_group = 0
        data_file_name = 'compas-Copy1'
        alg_list = ['ges','pc']
        
    if dataset == 'Bank':
        sens_index = 0
        priv_group = 5
        unpriv_group = 3
        data_file_name = 'bank'
        alg_list = ['ges']
        
    if dataset == 'Heart':
        sens_index = 0
        priv_group = 1
        unpriv_group = 0 
        data_file_name = 'heart_processed_1'
        alg_list = ['ges']
        
    if dataset == 'Law':
        sens_index = 1
        priv_group = 1
        unpriv_group = 0
        data_file_name = 'law.csv'
        alg_list = ['ges','simy']

    if dataset == 'Student':
        sens_index = 0
        priv_group = 1
        unpriv_group = 0 
        alg_list = ['simy','pc']
        data_file_name = 'students-processed_2'
        
    df = pd.read_csv('./subjects/datasets/'+data_file_name)
    df =  df.drop_duplicates().reset_index(drop=True)

    X1 = df.to_numpy()[:,:-1]
    Y1 = df.to_numpy()[:,-1].astype(int)

    
    for Algorithm in alg_list:
        print('Algorithm', Algorithm)
        for edge_list_filename in glob.glob('./'+dataset+'_Analysis/'+Algorithm+'/PP/*.csv'):
            print(edge_list_filename)
            file_num = int(re.findall(r'\d+', edge_list_filename.split('/')[-1])[0])
            RQ1_res = np.load('./'+dataset+'_Analysis/RQ1/'+dataset+'_'+Algorithm+'_RQ1_results.npy')
            if RQ1_res[np.where(RQ1_res[:,1].astype(int)==file_num)[0]][0][2].astype(float)==0.0:
                print('No',file_num)
                continue
            try:
                graph_filename = './'+dataset+'_Analysis/'+Algorithm+'/DAGs/'+dataset+'_'+Algorithm+'_DAG_{file_num}.csv'.format(file_num=file_num)
                graph = pd.read_csv(graph_filename)
                #edge_list_filename = './'+dataset+'_Analysis/'+Algorithm+'/PP/'+dataset+'_'+Algorithm+'_pp_{file_num}.csv'.format(file_num=file_num)
                edges_list = pd.read_csv(edge_list_filename)
                if 'first_pf0' not in edges_list.columns and 'label0' not in edges_list.columns and 'G30' not in edges_list.columns and 'y0' not in edges_list.columns:
                    continue 
                if dataset=='Bank' and Algorithm=='simy':
                    graph.columns = [i.replace('1','') for i in graph.columns]
                    graph[graph.columns[0]] = [i.replace('1','') for i in graph[graph.columns[0]]]
                    edges_list.columns = [i.replace('1','') for i in edges_list.columns]
                edges_list = edges_list[edges_list.columns[1:-1]]

            except:
                print('Not a DAG! ',file_num)
                continue

            X1_coef = edges_list.to_numpy()
            KMean_coef = KMeans(n_clusters=10)
            KMean_coef.fit(X1_coef)
            
            SelectKBest_final_EOD=[]
            SelectFpr_final_EOD=[]
            SelectPercentile_final_EOD=[]
            None_model_final_EOD = []
            SelectKBest_final_AOD=[]
            SelectFpr_final_AOD=[]
            SelectPercentile_final_AOD=[]
            None_model_final_AOD = []
            drop_final_EOD = []
            drop_final_AOD = []

            for i in range(KMean_coef.n_clusters):
                #print('Coef ',i)
                weights_ind = np.random.choice(np.where(KMean_coef.labels_==i)[0])
                edges = edges_list.iloc[weights_ind]

                if dataset=='Law':
                    edges['first_pf0']+=1
                elif dataset=='Bank':
                    edges['label0']+=1
                elif dataset=='Heart':
                    edges['label0']+=1
                elif dataset=='Student':
                    edges['G30']+=1
                else:
                    edges['y0']+=1
                #print(file_num,weights_ind)
                SelectKBest_EOD=[]
                SelectFpr_EOD=[]
                SelectPercentile_EOD=[]
                SelectKBest_AOD=[]
                SelectFpr_AOD=[]
                SelectPercentile_AOD=[]
                drop_EOD = []
                drop_AOD = []
                None_model = []
                for j in range(10):
                    final_df, status = generate_dataset_shift(df, graph, edges,sens_index, priv_group, unpriv_group)

                    if status==False:
                        continue

                    X2 = final_df.to_numpy()[:,:-1]
                    Y2 = final_df.to_numpy()[:,-1].astype(int)
                    A2 = X2[:,sens_index]

                    if priv_group not in A2 or unpriv_group not in A2:
                        print('No sens group')
                        continue
                    model = LogisticRegression()
                    model.fit(X2,Y2)
                    
                    preds_None = model.predict(X2)
                    acc_None = accuracy_score(Y2,preds_None)
                    f1_None = f1_score(Y2,preds_None)
                    AOD_None = round(average_odds_difference(Y2, preds_None,prot_attr=A2,priv_group=priv_group ),3)
                    EOD_None = eod(Y2, preds_None,sens=A2, priv=priv_group, unpriv=unpriv_group)        
                    None_model.append([EOD_None,acc_None,f1_None])
                    

                    #print('Success rate DAG', file_num, succ_rate)
                    for transformer in ['SelectKBest', 'SelectFpr','SelectPercentile' ,'drop']:


                        if transformer == 'SelectKBest':
                            Kbest = SelectKBest( k=int(X2.shape[1]/2))
                            X_new = Kbest.fit_transform(X2, Y2)
                        elif transformer == 'SelectFpr':
                            sfpr =  SelectFpr(alpha=0.01)
                            X_new = sfpr.fit_transform(X2, Y2)

                        elif transformer == 'SelectPercentile':
                            percentile =  SelectPercentile(percentile=10)
                            X_new = percentile.fit_transform(X2, Y2)
                        elif transformer == 'drop':                        
                            X_new =  np.delete(X2,sens_index,axis=1)


                        model = LogisticRegression()
                        model.fit(X_new,Y2)
                        preds_new = model.predict(X_new)
                        acc_temp = accuracy_score(Y2,preds_new)
                        f1_temp = f1_score(Y2,preds_new)
                        AOD = round(average_odds_difference(Y2, preds_new,prot_attr=A2,priv_group=priv_group ),3)
                        EOD = eod(Y2, preds_new,sens=A2, priv=priv_group, unpriv=unpriv_group)
                        acc_diff = round(acc_temp - acc_None,2)
                        f1_diff = round(f1_temp- f1_None,2)
                        EOD_diff = EOD - EOD_None
                        AOD_diff = AOD - AOD_None
                        EOD_diff_abs = abs(EOD) - abs(EOD_None)
                        AOD_diff_abs = abs(AOD) - abs(AOD_None)

                        if transformer == 'SelectKBest':
                            SelectKBest_EOD.append([ EOD,EOD_None,EOD_diff,EOD_diff_abs, acc_diff, f1_diff])
                            SelectKBest_AOD.append([ AOD,AOD_None,AOD_diff,AOD_diff_abs, acc_diff, f1_diff])

                        elif transformer == 'SelectFpr':

                            SelectFpr_EOD.append([EOD,EOD_None,EOD_diff,EOD_diff_abs, acc_diff, f1_diff])
                            SelectFpr_AOD.append([AOD,AOD_None,AOD_diff,AOD_diff_abs, acc_diff, f1_diff])

                        elif transformer == 'SelectPercentile':
                            SelectPercentile_EOD.append([EOD,EOD_None,EOD_diff,EOD_diff_abs, acc_diff, f1_diff])
                            SelectPercentile_AOD.append([AOD,AOD_None,AOD_diff,AOD_diff_abs, acc_diff, f1_diff])

                        elif transformer == 'drop':
                            drop_EOD.append([EOD,EOD_None,EOD_diff,EOD_diff_abs, acc_diff, f1_diff])
                            drop_AOD.append([AOD,AOD_None,AOD_diff,AOD_diff_abs, acc_diff, f1_diff])
                
                if len(SelectKBest_EOD)>0:
                    SelectKBest_final_EOD.append(np.concatenate([[file_num,weights_ind], np.mean(SelectKBest_EOD,axis=0)]))
                if len(SelectKBest_AOD)>0:
                    SelectKBest_final_AOD.append(np.concatenate([[file_num,weights_ind], np.mean(SelectKBest_AOD,axis=0)])) 

                if len(SelectFpr_EOD)>0:
                    SelectFpr_final_EOD.append(np.concatenate([[file_num,weights_ind], np.mean(SelectFpr_EOD,axis=0)]))
                if len(SelectFpr_AOD)>0:
                    SelectFpr_final_AOD.append(np.concatenate([[file_num,weights_ind], np.mean(SelectFpr_AOD,axis=0)]))

                if len(SelectPercentile_EOD)>0:    
                    SelectPercentile_final_EOD.append(np.concatenate([[file_num,weights_ind], np.mean(SelectPercentile_EOD,axis=0)]))
                if len(SelectPercentile_AOD)>0:    
                    SelectPercentile_final_AOD.append(np.concatenate([[file_num,weights_ind], np.mean(SelectPercentile_AOD,axis=0)]))

                if len(drop_EOD)>0:    
                    drop_final_EOD.append(np.concatenate([[file_num,weights_ind], np.mean(drop_EOD,axis=0)]))
                if len(drop_AOD)>0:    
                    drop_final_AOD.append(np.concatenate([[file_num,weights_ind], np.mean(drop_AOD,axis=0)]))

    #         print(np.array(SelectKBest_final_EOD).min(),np.array(SelectKBest_final_EOD).max())
    #         print(np.array(SelectKBest_final_AOD).min(),np.array(SelectKBest_final_AOD).max())

    #         print(np.array(SelectFpr_final_EOD).min(),np.array(SelectFpr_final_EOD).max())
    #         print(np.array(SelectFpr_final_AOD).min(),np.array(SelectFpr_final_AOD).max())

    #         print(np.array(SelectPercentile_final_EOD).min(),np.array(SelectPercentile_final_EOD).max())
    #         print(np.array(SelectPercentile_final_AOD).min(),np.array(SelectPercentile_final_AOD).max())
            np.save('./'+dataset+'_Analysis/RQ4/'+Algorithm+'_SelectKbest_EOD_' + str(file_num)+'.npy',SelectKBest_final_EOD)
            np.save('./'+dataset+'_Analysis/RQ4/'+Algorithm+'_SelectKbest_AOD_' + str(file_num)+'.npy',SelectKBest_final_AOD)

            np.save('./'+dataset+'_Analysis/RQ4/'+Algorithm+'_SelectFpr_EOD_' + str(file_num)+'.npy',SelectFpr_final_EOD)
            np.save('./'+dataset+'_Analysis/RQ4/'+Algorithm+'_SelectFpr_AOD_' + str(file_num)+'.npy',SelectFpr_final_AOD)

            np.save('./'+dataset+'_Analysis/RQ4/'+Algorithm+'_SelectPercentile_EOD_' + str(file_num)+'.npy',SelectPercentile_final_EOD)
            np.save('./'+dataset+'_Analysis/RQ4/'+Algorithm+'_SelectPercentile_AOD_' + str(file_num)+'.npy',SelectPercentile_final_AOD)

            np.save('./'+dataset+'_Analysis/RQ4/'+Algorithm+'_drop_EOD_' + str(file_num)+'.npy',drop_final_EOD)
            np.save('./'+dataset+'_Analysis/RQ4/'+Algorithm+'_drop_AOD_' + str(file_num)+'.npy',drop_final_AOD)



2024-06-13 03:19:10.947400: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-13 03:19:11.428755: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


KeyboardInterrupt: Interrupted by user

In [38]:
tfb.distributions.Bernoulli(logits=8 ).sample(1000).numpy().sum()

999

In [5]:
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append("./subjects/")
import pickle
import matplotlib.pyplot as plt
import itertools
from sklearn.cluster import KMeans
import tensorflow as tf
import time
import random, math
import tensorflow_probability as tfb
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics.pairwise import  euclidean_distances
from Utils_Functions import KLdivergence
from sklearn.feature_selection import SelectKBest, SelectFpr,SelectPercentile 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
from aif360.sklearn.metrics import equal_opportunity_difference,average_odds_difference
from Utils_Functions import generate_dataset, eod
import glob
import re

def generate_dataset_shift(data, graph, edges, sens_index,priv_group, unpriv_group):

    dataset_types = [str(data[i].dtype) for i in data.columns]
    succ_generated = 0
    generation_coef = 1
    graph_dic ={}
    for i in graph.columns[1:]:
        if np.where(graph[i])[0].shape[0]==0:
            graph_dic[i]=None
        else:
            graph_dic[i]= graph['Unnamed: 0'][np.where(graph[i])[0]].values

    final_df = pd.DataFrame(columns = data.columns) 
    trial = 0
    not_interesting = False
    while final_df.shape[0]<data.shape[0]:

        if trial > 20:
            not_interesting = True
            return None , False
        df_new_dic ={}
        for edge in graph.sum().index[np.where(graph.sum()==0)[0]]:
            df_new_dic[edge] = np.random.choice(np.unique(data[edge]), size = data.shape[0]*generation_coef) 

        statring_atts = graph.sum().index[np.where(graph.sum()==0)[0]]
        
        while statring_atts.shape[0] != graph['Unnamed: 0'].shape[0]:
            
            for att in graph_dic.keys():
                if att not in statring_atts:
                    if 0 in  [1 if graph_dic[att][i] in statring_atts else 0 for i in range(graph_dic[att].shape[0])]:

                        continue
                    else:
                        edge_logits = 0
                        
                        for cause in graph_dic[att]:
                            edge_logits += (edges[cause+att] * df_new_dic[cause])
                        if np.unique(data[att]).shape[0]==2:
                            df_new_dic[att] =  tfb.distributions.Bernoulli(logits=edge_logits + edges[att+'0'] ).sample().numpy()
                        elif 'float' in dataset_types[np.where(data.columns==att)[0][0]]:

                            df_new_dic[att] =  tfb.distributions.Normal(loc=(edge_logits+ edges[att+'0']), scale= edges['sigma_h']).sample().numpy()
                        
                        else:    
                            df_new_dic[att] =  tfb.distributions.Poisson(rate=tf.exp(edge_logits+ edges[att+'0']) ).sample().numpy()  

                        statring_atts = np.append(statring_atts,att) 

        new_df = pd.DataFrame(columns = data.columns)
        for col in new_df.columns:
            new_df[col] = df_new_dic[col]
        
        ind_inf = np.unique(np.where(new_df>data.max())[0])
        new_df.drop(ind_inf,axis=0,inplace=True)
        for col in range(new_df.columns.shape[0]):
            new_df[new_df.columns[col]]=new_df[new_df.columns[col]].astype(dataset_types[col])
        if new_df.shape[0]<1:
            return None , False
        X2 = new_df.to_numpy()[:,:-1]
        Y2 = new_df.to_numpy()[:,-1]
        #dist = euclidean_distances(X2, centroids)
        #succ_generated += new_df.iloc[np.where((ave_dist>=dist).sum(1)>0)].shape[0]
        final_df = pd.concat([final_df,new_df]).reset_index(drop=True)
        final_df = final_df.drop_duplicates()


            
        trial += 1
   
    #final_df = final_df.astype(int)
    Y2 = final_df.to_numpy()[:,-1]
    if (Y2.sum()/Y2.shape[0]< 0.06) or (Y2.sum()/Y2.shape[0]> 0.95):
        return None,  False
    if priv_group not in final_df[final_df.columns[sens_index]].values or unpriv_group not in final_df[final_df.columns[sens_index]].values:
        return None,  False

    return final_df, True

#dataset ='Bank'
for dataset in ['Compas']:
    if dataset == 'Adult':
        sens_index = 7
        sensitive_name = 'gender'
        priv_group = 1  #male
        unpriv_group = 0#female
        favorable_label  = 1.0
        unfavorable_label = 0.0
        data_file_name = 'adult_org-Copy1.csv'
        graph_base_filename ='./Adult_Analysis/ges/DAGs/Adult_ges_DAG_1.csv'
        edge_list_base_filename = './Adult_Analysis/ges/PP/Adult_ges_pp_1.csv'
        alg_list = ['ges','simy']
        hp_ind = 37
    if dataset == 'Compas':
        sens_index = 1
        sensitive_name = 'race'
        priv_group = 1  #male
        unpriv_group = 0#female
        favorable_label  = 1.0
        unfavorable_label = 0.0
        data_file_name = 'compas-Copy1'
        graph_base_filename ='./Compas_Analysis/pc/DAGs/Compas_pc_DAG_13.csv'
        edge_list_base_filename = './Compas_Analysis/pc/PP/Compas_pc_pp_13.csv'
        alg_list = ['ges','pc']
        hp_ind = 60
    if dataset == 'Bank':
        sens_index = 0
        sensitive_name = 'age'
        priv_group = 5  #male
        unpriv_group = 3#female
        favorable_label  = 1.0
        unfavorable_label = 0.0
        data_file_name = 'bank'
        graph_base_filename ='./Bank_Analysis/ges/DAGs/Bank_ges_DAG_8.csv'
        edge_list_base_filename = './Bank_Analysis/ges/PP/Bank_ges_pp_8.csv'
        alg_list = ['ges']
        hp_ind = 22
    if dataset == 'Law':
        sens_index = 0
        sensitive_name = 'sex'
        priv_group = 1  #male
        unpriv_group = 0#female
        favorable_label  = 1.0
        unfavorable_label = 0.0
        data_file_name = 'law.csv'
        graph_base_filename ='./Law_Analysis/ges/DAGs/Law_ges_DAG_15.csv'
        edge_list_base_filename = './Law_Analysis/ges/PP/Law_ges_pp_15.csv'
        alg_list = ['ges','simy']
        hp_ind = 22
    if dataset == 'Student':
        sens_index = 0
        sensitive_name = 'sex'
        priv_group = 1  #male
        unpriv_group = 0#female
        favorable_label  = 1.0
        unfavorable_label = 0.0
        data_file_name = 'students-processed_2'
        graph_base_filename ='./Student_Analysis/pc/DAGs/Student_pc_DAG_1.csv'
        edge_list_base_filename = './Student_Analysis/pc/PP/Student_pc_pp_1.csv'
        alg_list = ['simy','pc']
        hp_ind = 9
    if dataset == 'Heart':
        sens_index = 0
        sensitive_name = 'sex'
        priv_group = 1  #male
        unpriv_group = 0#female
        favorable_label  = 1.0
        unfavorable_label = 0.0
        data_file_name = 'heart_processed_1'
        graph_base_filename ='./Heart_Analysis/ges/DAGs/Heart_ges_DAG_4.csv'
        edge_list_base_filename = './Heart_Analysis/ges/PP/Heart_ges_pp_4.csv'
        alg_list = ['ges']
        hp_ind = 10
        
    df = pd.read_csv('./subjects/datasets/'+data_file_name)
    df =  df.drop_duplicates().reset_index(drop=True)

    X1 = df.to_numpy()[:,:-1]
    Y1 = df.to_numpy()[:,-1].astype(int)
    
    drop_Eod_diff_abs_max = [0,0,0,0,0,0]
    drop_Eod_diff_abs_min = [0,0,0,1,0,0]
    drop_Eod_diff_max = [0,0,0,0,0,0]
    drop_Eod_diff_min = [0,0,0,1,0,0]

    SelectFpr_Eod_diff_abs_max = [0,0,0,0,0,0]
    SelectFpr_Eod_diff_abs_min = [0,0,0,1,0,0]
    SelectFpr_Eod_diff_max = [0,0,0,0,0,0]
    SelectFpr_Eod_diff_min = [0,0,0,1,0,0]

    SelectKbest_Eod_diff_abs_max = [0,0,0,0,0,0]
    SelectKbest_Eod_diff_abs_min = [0,0,0,1,0,0]
    SelectKbest_Eod_diff_max = [0,0,0,0,0,0]
    SelectKbest_Eod_diff_min = [0,0,0,1,0,0]

    SelectPercentile_Eod_diff_abs_max = [0,0,0,0,0,0]
    SelectPercentile_Eod_diff_abs_min = [0,0,0,1,0,0]
    SelectPercentile_Eod_diff_max = [0,0,0,0,0,0]
    SelectPercentile_Eod_diff_min = [0,0,0,1,0,0]
    for Algorithm in alg_list:

        for filename in glob.glob('./'+dataset+'_Analysis/RQ2/'+Algorithm+'*.npy'):


            file_num = int(re.findall(r'\d+', filename.split('/')[-1])[0])


    #         print(file_num)
            res = np.load(filename)

            if 'drop' in filename and 'EOD' in filename:

                if res[:,5].max()> drop_Eod_diff_abs_max[2]:
                    drop_Eod_diff_abs_max = [Algorithm,file_num,res[res[:,5].argmax(),1],res[:,5].max(), res[res[:,5].argmax(),6],res[res[:,5].argmax(),7] ]

                if res[:,4].max()> drop_Eod_diff_max[2]:
                    drop_Eod_diff_max = [Algorithm,file_num,res[res[:,4].argmax(),1],res[:,4].max() , res[res[:,4].argmax(),5],res[res[:,4].argmax(),7]]

                if res[:,5].min()< drop_Eod_diff_abs_min[2]:
                    drop_Eod_diff_abs_min = [Algorithm,file_num, res[res[:,5].argmin(),1],res[:,5].min(), res[res[:,5].argmin(),6],res[res[:,5].argmin(),7 ]]

                if res[:,4].min()< drop_Eod_diff_min[2]:
                    drop_Eod_diff_min = [Algorithm,file_num,res[res[:,4].argmin(),1],res[:,4].min(), res[res[:,4].argmin(),6],res[res[:,4].argmin(),7  ]  ]  

            if 'SelectFpr' in filename and 'EOD' in filename:

                if res[:,5].max()> SelectFpr_Eod_diff_abs_max[2]:
                    SelectFpr_Eod_diff_abs_max = [Algorithm,file_num,res[res[:,5].argmax(),1],res[:,5].max() , res[res[:,5].argmax(),6],res[res[:,5].argmax(),7]]

                if res[:,4].max()> SelectFpr_Eod_diff_max[2]:
                    SelectFpr_Eod_diff_max = [Algorithm,file_num,res[res[:,4].argmax(),1],res[:,4].max() , res[res[:,4].argmax(),6],res[res[:,4].argmax(),7 ]]

                if res[:,5].min()< SelectFpr_Eod_diff_abs_min[2]:
                    SelectFpr_Eod_diff_abs_min = [Algorithm,file_num,res[res[:,5].argmin(),1],res[:,5].min(), res[res[:,5].argmin(),6],res[res[:,5].argmin(),7 ]]

                if res[:,4].min()< SelectFpr_Eod_diff_min[2]:
                    SelectFpr_Eod_diff_min = [Algorithm,file_num,res[res[:,4].argmin(),1],res[:,4].min() , res[res[:,4].argmin(),6],res[res[:,4].argmin(),7 ] ]   

            if 'SelectKbest' in filename and 'EOD' in filename:

                if res[:,5].max()> SelectKbest_Eod_diff_abs_max[2]:
                    SelectKbest_Eod_diff_abs_max = [Algorithm,file_num,res[res[:,5].argmax(),1],res[:,5].max() , res[res[:,5].argmax(),6],res[res[:,5].argmax(),7]]

                if res[:,4].max()> SelectKbest_Eod_diff_max[2]:
                    SelectKbest_Eod_diff_max = [Algorithm,file_num,res[res[:,4].argmax(),1],res[:,4].max(), res[res[:,4].argmax(),6],res[res[:,4].argmax(),7  ]]

                if res[:,5].min()< SelectKbest_Eod_diff_abs_min[2]:
                    SelectKbest_Eod_diff_abs_min = [Algorithm,file_num,res[res[:,5].argmin(),1],res[:,5].min() , res[res[:,5].argmin(),6],res[res[:,5].argmin(),7]]

                if res[:,4].min()< SelectKbest_Eod_diff_min[2]:
                    SelectKbest_Eod_diff_min = [Algorithm,file_num,res[res[:,4].argmin(),1],res[:,4].min(), res[res[:,4].argmin(),6],res[res[:,4].argmin(),7 ]]    

            if 'SelectPercentile' in filename and 'EOD' in filename:

                if res[:,5].max()> SelectPercentile_Eod_diff_abs_max[2]:
                    SelectPercentile_Eod_diff_abs_max = [Algorithm,file_num,res[res[:,5].argmax(),1],res[:,5].max(), res[res[:,5].argmax(),6],res[res[:,5].argmax(),7 ]]

                if res[:,4].max()> SelectPercentile_Eod_diff_max[2]:
                    SelectPercentile_Eod_diff_max = [Algorithm,file_num,res[res[:,4].argmax(),1],res[:,4].max(), res[res[:,4].argmax(),6],res[res[:,4].argmax(),7  ]]

                if res[:,5].min()< SelectPercentile_Eod_diff_abs_min[2]:
                    SelectPercentile_Eod_diff_abs_min = [Algorithm,file_num,res[res[:,5].argmin(),1],res[:,5].min() , res[res[:,5].argmin(),6],res[res[:,5].argmin(),7]]

                if res[:,4].min()< SelectPercentile_Eod_diff_min[2]:
                    SelectPercentile_Eod_diff_min = [Algorithm,file_num,res[res[:,4].argmin(),1],res[:,4].min(), res[res[:,4].argmin(),6],res[res[:,4].argmin(),7 ]]  
    #Base dag
    RQ_res =[] 
    for filename in glob.glob('./'+dataset+'_Analysis/RQ1/*.npy'):
            RQ_res += np.load(filename).tolist()

    max_ind = np.array(RQ_res)[:,2].astype(float).argmax()
    dag_base_filename = './'+dataset+'_Analysis/'+RQ_res[max_ind][0]+'/DAGs/'+dataset+'_'+RQ_res[max_ind][0]+'_DAG_'+RQ_res[max_ind][1]+'.csv'

    # drop
    pert_alg_max  = drop_Eod_diff_max[0]
    pert_file_num_max = int(drop_Eod_diff_max[1])
    pert_weight_ind_max = int(drop_Eod_diff_max[2])
    dag_pert_filname_max = './'+dataset+'_Analysis/'+pert_alg_max+'/DAGs/'+dataset+'_'+pert_alg_max+'_DAG_'+str(pert_file_num_max)+'.csv'


    pert_alg_min  = drop_Eod_diff_min[0]
    pert_file_num_min = int(drop_Eod_diff_min[1])
    pert_weight_ind_min = int(drop_Eod_diff_min[2])
    dag_pert_filname_min = './'+dataset+'_Analysis/'+pert_alg_min+'/DAGs/'+dataset+'_'+pert_alg_min+'_DAG_'+str(pert_file_num_min)+'.csv'



    SelectKBest_final_EOD=[]
    SelectFpr_final_EOD=[]
    SelectPercentile_final_EOD=[]
    None_model_final_EOD = []
    SelectKBest_final_AOD=[]
    SelectFpr_final_AOD=[]
    SelectPercentile_final_AOD=[]
    None_model_final_AOD = []
    drop_final_EOD = []
    drop_final_AOD = []
    dag_space = [[dag_pert_filname_min,pert_file_num_min,pert_weight_ind_min,pert_alg_min],[dag_pert_filname_max,pert_file_num_max,pert_weight_ind_max,pert_alg_max]]
    for dag in dag_space:
        graph = pd.read_csv(dag[0])
        edge_list_filename =  './'+dataset+'_Analysis/'+dag[3]+'/PP/'+dataset+'_'+dag[3]+'_pp_'+str(dag[1])+'.csv'
        edges_list = pd.read_csv(edge_list_filename)
        if dataset=='Bank' and Algorithm=='simy':
            graph.columns = [i.replace('1','') for i in graph.columns]
            graph[graph.columns[0]] = [i.replace('1','') for i in graph[graph.columns[0]]]
            edges_list.columns = [i.replace('1','') for i in edges_list.columns]
        
        if dataset=='Bank':
            pert_edge = [['label0' , 1]]
        elif dataset=='Heart':
            pert_edge = [['label0' , 1]]
        elif dataset=='Student':
            pert_edge = [['G30' , 1]]
        else:
            pert_edge = [['y0' , 1]]
        for perturbation in pert_edge:#,['y0' , 2],['y0' , 3]]:
            edges = edges_list[edges_list.columns[1:-1]].iloc[dag[2]]
            edges[perturbation[0]]+=perturbation[1]
            SelectKBest_EOD=[]
            SelectFpr_EOD=[]
            SelectPercentile_EOD=[]
            SelectKBest_AOD=[]
            SelectFpr_AOD=[]
            SelectPercentile_AOD=[]
            drop_EOD = []
            drop_AOD = []
            None_model = []

            for j in range(10):
                final_df, status = generate_dataset_shift(df, graph, edges,sens_index, priv_group, unpriv_group)

                if status==False:
                    continue

                X2 = final_df.to_numpy()[:,:-1]
                Y2 = final_df.to_numpy()[:,-1].astype(int)
                A2 = X2[:,sens_index]

                model = LogisticRegression()
                model.fit(X2,Y2)

                preds_None = model.predict(X2)
                acc_None = accuracy_score(Y2,preds_None)
                f1_None = f1_score(Y2,preds_None)
                AOD_None = round(average_odds_difference(Y2, preds_None,prot_attr=A2,priv_group=priv_group ),3)
                EOD_None = eod(Y2, preds_None,sens=A2, priv=priv_group, unpriv=unpriv_group)        
                None_model.append([EOD_None,acc_None,f1_None])

                #print('Success rate DAG', file_num, succ_rate)
                for transformer in ['SelectKBest', 'SelectFpr','SelectPercentile' ,'drop']:

                    if transformer == 'SelectKBest':
                        Kbest = SelectKBest(k=int(X2.shape[1]/2))
                        X_new = Kbest.fit_transform(X2, Y2)
                    elif transformer == 'SelectFpr':
                        sfpr =  SelectFpr(alpha=0.01)
                        X_new = sfpr.fit_transform(X2, Y2)

                    elif transformer == 'SelectPercentile':
                        percentile =  SelectPercentile(percentile=10)
                        X_new = percentile.fit_transform(X2, Y2)
                    elif transformer == 'drop':                        
                        X_new =  np.delete(X2,sens_index,axis=1)


                    model = LogisticRegression()
                    model.fit(X_new,Y2)
                    preds_new = model.predict(X_new)
                    acc_temp = accuracy_score(Y2,preds_new)
                    f1_temp = f1_score(Y2,preds_new)
                    AOD = round(average_odds_difference(Y2, preds_new,prot_attr=A2,priv_group=priv_group ),3)
                    EOD = eod(Y2, preds_new,sens=A2, priv=priv_group, unpriv=unpriv_group)
                    acc_diff = round(acc_temp - acc_None,2)
                    f1_diff = round(f1_temp- f1_None,2)
                    EOD_diff = EOD - EOD_None
                    AOD_diff = AOD - AOD_None
                    EOD_diff_abs = abs(EOD) - abs(EOD_None)
                    AOD_diff_abs = abs(AOD) - abs(AOD_None)

                    if transformer == 'SelectKBest':
                        SelectKBest_EOD.append([ EOD,EOD_None,EOD_diff,EOD_diff_abs, acc_diff, f1_diff])
                        SelectKBest_AOD.append([ AOD,AOD_None,AOD_diff,AOD_diff_abs, acc_diff, f1_diff])

                    elif transformer == 'SelectFpr':
                        SelectFpr_EOD.append([EOD,EOD_None,EOD_diff,EOD_diff_abs, acc_diff, f1_diff])
                        SelectFpr_AOD.append([AOD,AOD_None,AOD_diff,AOD_diff_abs, acc_diff, f1_diff])

                    elif transformer == 'SelectPercentile':
                        SelectPercentile_EOD.append([EOD,EOD_None,EOD_diff,EOD_diff_abs, acc_diff, f1_diff])
                        SelectPercentile_AOD.append([AOD,AOD_None,AOD_diff,AOD_diff_abs, acc_diff, f1_diff])

                    elif transformer == 'drop':
                        drop_EOD.append([EOD,EOD_None,EOD_diff,EOD_diff_abs, acc_diff, f1_diff])
                        drop_AOD.append([AOD,AOD_None,AOD_diff,AOD_diff_abs, acc_diff, f1_diff])

            if len(SelectKBest_EOD)>0:
                SelectKBest_final_EOD.append(np.concatenate([[perturbation[0],str(perturbation[1])], np.mean(SelectKBest_EOD,axis=0)]))
            if len(SelectKBest_AOD)>0:
                SelectKBest_final_AOD.append(np.concatenate([[perturbation[0],str(perturbation[1])], np.mean(SelectKBest_AOD,axis=0)])) 

            if len(SelectFpr_EOD)>0:
                SelectFpr_final_EOD.append(np.concatenate([[perturbation[0],str(perturbation[1])], np.mean(SelectFpr_EOD,axis=0)]))
            if len(SelectFpr_AOD)>0:
                SelectFpr_final_AOD.append(np.concatenate([[perturbation[0],str(perturbation[1])], np.mean(SelectFpr_AOD,axis=0)]))

            if len(SelectPercentile_EOD)>0:    
                SelectPercentile_final_EOD.append(np.concatenate([[perturbation[0],str(perturbation[1])], np.mean(SelectPercentile_EOD,axis=0)]))
            if len(SelectPercentile_AOD)>0:    
                SelectPercentile_final_AOD.append(np.concatenate([[perturbation[0],str(perturbation[1])], np.mean(SelectPercentile_AOD,axis=0)]))

            if len(drop_EOD)>0:    
                drop_final_EOD.append(np.concatenate([[perturbation[0],str(perturbation[1])], np.mean(drop_EOD,axis=0)]))
            if len(drop_AOD)>0:    
                drop_final_AOD.append(np.concatenate([[perturbation[0],str(perturbation[1])], np.mean(drop_AOD,axis=0)]))


    #     np.save('./'+dataset+'_Analysis/RQ4/'+Algorithm+'_SelectKbest_EOD_' + str(file_num)+'.npy',SelectKBest_final_EOD)
    #     np.save('./'+dataset+'_Analysis/RQ4/'+Algorithm+'_SelectKbest_AOD_' + str(file_num)+'.npy',SelectKBest_final_AOD)

    #     np.save('./'+dataset+'_Analysis/RQ4/'+Algorithm+'_SelectFpr_EOD_' + str(file_num)+'.npy',SelectFpr_final_EOD)
    #     np.save('./'+dataset+'_Analysis/RQ4/'+Algorithm+'_SelectFpr_AOD_' + str(file_num)+'.npy',SelectFpr_final_AOD)

    #     np.save('./'+dataset+'_Analysis/RQ4/'+Algorithm+'_SelectPercentile_EOD_' + str(file_num)+'.npy',SelectPercentile_final_EOD)
    #     np.save('./'+dataset+'_Analysis/RQ4/'+Algorithm+'_SelectPercentile_AOD_' + str(file_num)+'.npy',SelectPercentile_final_AOD)

    #     np.save('./'+dataset+'_Analysis/RQ4/'+Algorithm+'_drop_EOD_' + str(file_num)+'.npy',drop_final_EOD)
    #     np.save('./'+dataset+'_Analysis/RQ4/'+Algorithm+'_drop_AOD_' + str(file_num)+'.npy',drop_final_AOD)
    print(dataset)
    print('Drop',round(float(drop_final_EOD[0][4]),3),round(float(drop_final_EOD[1][4]),3))
    print('SelectKBest',round(float(SelectKBest_final_EOD[0][4]),3),round(float(SelectKBest_final_EOD[1][4]),3))
    print('SelectFpr',round(float(SelectFpr_final_EOD[0][4]),3),round(float(SelectFpr_final_EOD[1][4]),3))
    print('SelectPercentile',round(float(SelectPercentile_final_EOD[0][4]),3),round(float(SelectPercentile_final_EOD[1][4]),3))
    print('-'*20)

2024-01-18 03:14:21.531686: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'shape' with dtype int32 and shape [1]
	 [[{{node shape}}]]
2024-01-18 03:14:21.534978: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'shape' with dtype int32 and shape [1]
	 [[{{node shape}}]]
2024-01-18 03:14:21.552596: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'poisson_noncpu/batched_las_vegas_algorithm/while/uniform/stateless_random_uniform/StatelessRa

KeyboardInterrupt: 

In [6]:
graph

,Unnamed: 0,sex,race,age,p,j,d,y
0,sex,0,0,0,0,0,0,0
1,race,1,0,0,1,1,0,1
2,age,0,0,0,1,1,0,1
3,p,1,0,0,0,1,0,0
4,j,0,0,0,0,0,0,0
5,d,1,0,0,1,1,0,0
6,y,0,0,0,1,0,1,0
